<a href="https://colab.research.google.com/github/yneha70/IIITH/blob/main/Task_2_Performance_Evalution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from ultralytics import YOLO
import cv2
import os
import numpy as np
import m
image_folder = r"C:\Users\Neha\Desktop\YOLO_Restaurant_Project\images"
output_folder = r"C:\Users\Neha\Desktop\YOLO_Restaurant_Project\outputs"
os.makedirs(output_folder, exist_ok=True)
metrics_folder = os.path.join(output_folder, "performance_metrics")
os.makedirs(metrics_folder, exist_ok=True)

model = YOLO("yolov8l.pt")

target_objects = {
    "Restaurant.jpg": "person",
    "Train.jpg": "train",
    "Dog.jpg": "dog",
    "Cycling.jpg": "bicycle"
}

image_names = []
f1_scores = []
precisions = []
recalls = []

class_labels = ["person", "train", "dog", "bicycle"]
cm = np.zeros((4, 4), dtype=int)

BOUNDARY_THICKNESS = 2
FONT_SCALE = 0.45
FONT_THICKNESS = 1
LABEL_BG_ALPHA = 0.7

for image_name in os.listdir(image_folder):
    if not image_name.lower().endswith((".jpg", ".png", ".jpeg")):
        continue

    image_path = os.path.join(image_folder, image_name)
    img = cv2.imread(image_path)
    if img is None:
        print(f"⚠️ Skipping unreadable image: {image_name}")
        continue

    results = model.predict(source=image_path, conf=0.25, imgsz=1280, device="cpu", save=False)
    boxes = results[0].boxes
    detected_classes = [model.names[int(c)].lower() for c in boxes.cls]

    true_label = target_objects.get(image_name, None)
    if not true_label:
        continue

    for det_class in detected_classes:
        if det_class in class_labels:
            true_idx = class_labels.index(true_label)
            pred_idx = class_labels.index(det_class)
            cm[true_idx, pred_idx] += 1

    TP = detected_classes.count(true_label)
    FP = len(detected_classes) - TP
    FN = 1 if TP == 0 else 0
    precision = TP / (TP + FP + 1e-6)
    recall = TP / (TP + FN + 1e-6)
    f1 = 2 * precision * recall / (precision + recall + 1e-6)

    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)
    image_names.append(image_name.split(".")[0])

    for box, cls in zip(boxes.xyxy, boxes.cls):
        class_name = model.names[int(cls)].lower()
        if class_name not in ["person", "train", "dog", "bicycle"]:
            continue
        conf = float(boxes.conf[0])
        xyxy = [int(x) for x in box.tolist()]
        color = (0, 255, 0)
        label = f"{class_name.capitalize()} {conf:.2f}"
        cv2.rectangle(img, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), color, BOUNDARY_THICKNESS)
        cv2.putText(img, label, (xyxy[0], xyxy[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, FONT_SCALE, (0, 0, 0), FONT_THICKNESS)

    cv2.imwrite(os.path.join(output_folder, f"detected_{image_name}"), img)

plt.figure(figsize=(10, 6))
plt.plot(image_names, f1_scores, marker='o', label="F1 Score", color='blue')
plt.plot(image_names, precisions, marker='o', label="Precision", color='green')
plt.plot(image_names, recalls, marker='o', label="Recall", color='red')
plt.xlabel("Images")
plt.ylabel("Metric Value")
plt.title("Performance Metrics Comparison")
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(metrics_folder, "performance_metrics_combined.png"))
plt.close()

plt.figure(figsize=(6, 6))
plt.imshow(cm, cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.xticks(np.arange(len(class_labels)), class_labels)
plt.yticks(np.arange(len(class_labels)), class_labels)

for i in range(len(class_labels)):
    for j in range(len(class_labels)):
        plt.text(j, i, str(cm[i, j]), ha='center', va='center', color='black')

plt.tight_layout()
plt.savefig(os.path.join(metrics_folder, "confusion_matrix.png"))
plt.close()

print("\n Detection complete. All outputs and graphs saved in:")
print(metrics_folder)
